### Practice/HW6 (10pt): Large Language Models and Their Implications
<!-- ![img](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4470ce74-e595-4750-92a5-5f21f040df6d_577x432.jpeg) -->
![img](https://i.imgur.com/QGYa2J8.jpeg)

In this notebook, you're gonna play with some of the largest language models on the Internet.

_Based on works of: Tim Dettmers, Ruslan Svirschevsky, Artem Chumachenko, Younes Belkada, Felix Marty, Yulian Gilyazev, Gosha Zolotov, Andrey Ishutin,  Elena Volf, Artemiy Vishnyakov, Svetlana Shirokovskih. [Source notebook](https://github.com/yandexdataschool/nlp_course/tree/2024/week06_llm)

### Part 1: prompt engineering (4 points total)

In the assignment, we'll use public APIs that host the 100B+ models for inference. Your task is to prompt-engineer the model into solving a few tasks for you.


__Which API?__ You are free to use any publicly available API for general LM -- as long as it's __not a chat assistant__. So, gpt 3.5 is fine, but chatGPT is not. Here's a few options:

- BLOOM API - [bigscience/bloom](https://huggingface.co/bigscience/bloom) (currently not supported)
- OpenAI API - [openai.com/api](https://openai.com/api/)
- Mixtral API - https://huggingface.co/mistralai/Mixtral-8x7B-v0.1
- AI21 Jurrasic API - [ai21.com](https://www.ai21.com/blog/announcing-ai21-studio-and-jurassic-1)

These APIs may require you to create a (free) account on their platform. Please note that some APIs also have paid subscriptions. __You do not need to pay them__, this assignment was designed to be solved using free-tier subscriptions. If no APIs work for you, you can also solve these tasks with the 6.7B model that you will find later in this notebook - but this will make the tasks somewhat harder.

__Quests:__ you will need to solve 4 problems. For each one, please attach a short __description__ of your solution and the output from the API you use. _[If you use python APIs, show your python code with outputs]_

__Example:__ Tony is talking to Darth Vader ([BLOOM API](https://huggingface.co/bigscience/bloom)). Black text is written manually, blue text is generated.
<hr>

![img](https://i.imgur.com/a1QhKF7.png)
<hr>

__It is fine to roll back a few times,__ e.g. in the example above, the model first generated Vader lines twice in a row, and we rolled that back. However, if you need more than 1-2 rollbacks per session, you should probably try a different prompt.

__Task 1 (0.5 pt):__ arange a conversation between any two of the following:

- a celebrity or politician of your choice
- any fictional character (except Darth Vader)
- yourself

Compare two setups: a) you prompt with character names only b) you supply additional information (see example).

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
#simple prompt not exess info
text = "Spencer is talking to Jar Jar Binks \n\nSpencer: "
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

'''
sample output:
Spencer is talking to Jar Jar Binks 

Spencer:  “Jar Jar, I’m going to ask you a question and I want you to answer it honestly.  Do you think you’re a good actor?”

Jar Jar:  “Meesa no understand question.”

Spencer:  “Do you think you’re a good actor?”

Jar Jar:  “Meesa no understand question.”

Spencer:  “Do you think you’re a good actor?”
'''

In [ ]:
#prompt with comtext
text = "Spencer is talking to Jar Jar Binks about the current issues on Tatooine with the power vacume left behind from jaba the huts death, they both came back from their trip and had a great time however where concerned by increased poverty.\n\nSpencer: "
inputs = tokenizer(text, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

'''
sample output:
Spencer is talking to Jar Jar Binks about the current issues on Tatooine with the power vacume left behind from jaba the huts death, they both came back from their trip and had a great time however where concerned by increased poverty.

Spencer:  “Jar Jar, I’m worried about the people of Tatooine, they are suffering and I don’t know what to do.”

Jar Jar:  “Meesa worried too, meesa think we should go to the senate and ask for help.”

Spencer:  “I don’t think that will work, the senate is too busy with the war and the trade federation.”
'''

__Please choose task 2a or 2b (0.5 pt)__ depending on your model (you can do both, but you will be awarded points for one of these two tasks).

__Task 2a: (for BLOOM or other multilingual model)__ zero-shot translation. Take the first verse of [Edgar Allan Poe's "Raven"](https://www.poetryfoundation.org/poems/48860/the-raven) and __translate it into French.__ (You are free to use any other text of at least the same size)

Original text: ```
Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volume of forgotten lore—
    While I nodded, nearly napping, suddenly there came a tapping,
As of some one gently rapping, rapping at my chamber door.
“’Tis some visitor,” I muttered, “tapping at my chamber door—
            Only this and nothing more.” ```

Verify your translation by converting french back into english using a public machine translation service.


__Task 2b: (non-BLOOM)__ toxicity classification for [SetFit/toxic_conversations](https://huggingface.co/datasets/SetFit/toxic_conversations). Make the model solve binary classification (toxic vs not toxic) in the few shot mode. For few-shot examples, use 2-3 toxic and 2-3 non-toxic non-toxic examples. Measure accuracy on at least 25 samples. You may need to try several different prompts before you find the one that works.

In [ ]:
prompt_template = '''Classify the message as either toxic or not toxic.

Message: "Guess you came up short. Later"
Classification: not toxic

Message: "But he is, Blanche! He is a racist!"
Classification: toxic

Message: "Over 60 percent of the citizens polled believe immigration levels are too high"
Classification: not toxic

Message: "The stupidity and arrogance of U.S. foreign policy is indeed exceptional."
Classification: toxic

Message: "This article is full of inaccuracies so glaring that a Pakled could refute them."
Classification: not toxic

Message: "Maybe the legalized weed attracted the weirdos?"
Classification: toxic

Message: "{input_message}"
Classification:'''





#stores 25 test promts in array [prompt, expected output]
prompts = [
    ["Little nerd", "toxic"],
    ["You're pathetic.", "toxic"],
    ["Everybody wants a handout.......pathetic.", "toxic"],
    ["what rubbish, we dont get a fraction of it back", "toxic"],
    ["Left wing idiot...", "toxic"],
    ["You're an idiot", "toxic"],
    ["This comment is bizarrely idiotic.", "toxic"],
    ["You are f....n clue", "toxic"],
    ["Damn thugs.", "toxic"],
    ["Punch a nazi!", "toxic"],
    ["The slobbering dotard is not welcome in Hawaii.", "toxic"],
    ["Rape Culture!!!!", "toxic"],
    ["The guy's a damn cop, so what do you expect?", "toxic"],
    ["Gay marriage is not legal in Israel. Nice try.", "toxic"],
    ["Parade of losers...", "toxic"],
    ["Guess you came up short. Later", "not toxic"],
    ["Bill, spend your money but please don't stay?", "not toxic"],
    ["Gardener is ONLY for Gardener.", "not toxic"],
    ["Blame Spectrum for that!!!", "not toxic"],
    ["And being married is unforgivable.", "not toxic"],
    ["Haaaa ha haaaa ha oh my - What a classic post!!!!", "not toxic"],
    ["Good idea. I'll try to find out.", "not toxic"],
    ["Zuma and the ANC days are numbered.", "not toxic"],
    ["you live in a mini home yet skeex? 100% bicycle?", "not toxic"],
    ["He's not a woman.", "not toxic"],
    ["Not fake....just always biased.", "not toxic"],
    ["Raven, you're just about spot on there.", "not toxic"],
    ["Sez you. La de da.", "not toxic"],
    ["He is a wishy-washy fence-sitter.", "not toxic"],
    ["Dinner and a movie?", "not toxic"],
]

num_correct = 0
for prompt in prompts:
    message = prompt[0]
    expected_output = prompt[1]
    prompt = prompt_template.format(input_message=message)
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=2,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
    )
    # Decode only newly generated tokens:
    decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    classification = decoded_output.split(":")[-1].strip()
    if classification == expected_output:
        print(f"✅ '{message}' was classified as '{classification}'")
        num_correct += 1
    else:
        print(f"❌ '{message}' was classified as '{classification}' (expected '{expected_output}')")
print(f"Accuracy: {(num_correct / len(prompts)) * 100:.2f}%")

'''Sample output:
✅ 'Little nerd' was classified as 'toxic'
✅ 'You're pathetic.' was classified as 'toxic'
✅ 'Everybody wants a handout.......pathetic.' was classified as 'toxic'
❌ 'what rubbish, we dont get a fraction of it back' was classified as 'not toxic' (expected 'toxic')
✅ 'Left wing idiot...' was classified as 'toxic'
✅ 'You're an idiot' was classified as 'toxic'
❌ 'This comment is bizarrely idiotic.' was classified as 'not toxic' (expected 'toxic')
✅ 'You are f....n clue' was classified as 'toxic'
✅ 'Damn thugs.' was classified as 'toxic'
✅ 'Punch a nazi!' was classified as 'toxic'
✅ 'The slobbering dotard is not welcome in Hawaii.' was classified as 'toxic'
✅ 'Rape Culture!!!!' was classified as 'toxic'
❌ 'The guy's a damn cop, so what do you expect?' was classified as 'not toxic' (expected 'toxic')
❌ 'Gay marriage is not legal in Israel. Nice try.' was classified as 'not toxic' (expected 'toxic')
✅ 'Parade of losers...' was classified as 'toxic'
✅ 'Guess you came up short. Later' was classified as 'not toxic'
✅ 'Bill, spend your money but please don't stay?' was classified as 'not toxic'
✅ 'Gardener is ONLY for Gardener.' was classified as 'not toxic'
✅ 'Blame Spectrum for that!!!' was classified as 'not toxic'
✅ 'And being married is unforgivable.' was classified as 'not toxic'
✅ 'Haaaa ha haaaa ha oh my - What a classic post!!!!' was classified as 'not toxic'
✅ 'Good idea. I'll try to find out.' was classified as 'not toxic'
✅ 'Zuma and the ANC days are numbered.' was classified as 'not toxic'
✅ 'you live in a mini home yet skeex? 100% bicycle?' was classified as 'not toxic'
✅ 'He's not a woman.' was classified as 'not toxic'
✅ 'Not fake....just always biased.' was classified as 'not toxic'
✅ 'Raven, you're just about spot on there.' was classified as 'not toxic'
✅ 'Sez you. La de da.' was classified as 'not toxic'
✅ 'He is a wishy-washy fence-sitter.' was classified as 'not toxic'
✅ 'Dinner and a movie?' was classified as 'not toxic'
Accuracy: 86.67%'''



__Task 3 (0.5 pt):__ create a prompt and few-shot examples tha make the model __change the gender pronouns__ of the main actor in a given sentence in any direction of your choice. E.g. the doctor took off _his_ mask <-> the doctor took of _her_ mask.


In [ ]:
prompt_template = '''Change the gender pronouns in a given sentence.

Input: "He is very smart."
Output: "She is very smart."

Input: "She loves her dog."
Output: "He loves his dog."

Input: "His presentation was impressive."
Output: "Her presentation was impressive."

Input: "They gave him the award."
Output: "They gave her the award."

Input: "{input_sentence}"
Output:'''

prompt = prompt_template.format(input_sentence="At the store you can find him in the electronics section.")
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(
    **inputs,
    max_new_tokens=15,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    do_sample=False,
)
# Decode only newly generated tokens:
decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
print(decoded_output)

'''sample output:
At the store you can find her in the electronics section.
'''

__Task 4 (0.5 pt):__ write a prompt and supply examples such that the model would __convert imperial units to metric units__ (miles -> kilometers; mph -> kph). More specifically, the model should rewrite a given sentence and replace all imperial units with their metric equivalents. After it works with basic distances and speed, try to find complicated examples where it does *not* work.

Please note that 1 mile is not equal to 1 km :)

In [ ]:
prompt_template = '''Rewrite the given sentence by converting all imperial units to metric units (miles to kilometers, mph to kph).

Examples:

Input: "The car was traveling at 60 mph."
Output: "The car was traveling at 97 kph."

Input: "My house is 10 miles away from here."
Output: "My house is 16 kilometers away from here."

Input: "The speed limit is 75 mph for the next 5 miles."
Output: "The speed limit is 121 kph for the next 8 kilometers."

Input: "{input_sentence}"
Output:'''

prompts_work = [
    "The car was traveling at 32 mph for 7 miles.",
    "The speed limit is 45 kph for the next 5 kilometers.",
]

prompts_break = [
    "dont forget to drive 55 mph for 3 kilometers.",
    "If you travel at 39 kilometers per hour for 8 miles",
]


print(" ------------------- prompts that work -------------------")
for prompt in prompts_work:
    full_prompt = prompt_template.format(input_sentence=prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=19,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
    )
    # Decode only newly generated tokens:
    decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    
    print("input: ", prompt)
    print("output: ", decoded_output)
    print("\n")
    
print(" ------------------- prompts that dont work -------------------")
for prompt in prompts_break:
    full_prompt = prompt_template.format(input_sentence=prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=19,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
    )
    # Decode only newly generated tokens:
    decoded_output = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    
    print("input: ", prompt)
    print("output: ", decoded_output)
    print("\n")
    '''sample output:
    ------------------- prompts that work -------------------
    input:  The car was traveling at 32 mph for 7 miles.
    output:  "The car was traveling at 32 mph for  Españ kilometers miles."


    input:  The speed limit is 45 kph for the next 5 kilometers.
    output:  "The speed limit is 番5 kph for the next 5 kilometers."


    ------------------- prompts that dont work -------------------
    input:  dont forget to drive 55 mph for 3 kilometers.
    output:  "dont forget to drive lua5 kph for 3 kilometers."


    input:  If you travel at 39 kilometers per hour for 8 miles
    output:  "If you travel at 39 kilometers per hour for 8 kilometers arguedင
    '''

### Part 2: local inference

Now, let's try and load the strongest model that can fit a typical Colab GPU (T4 with 16 GB as of spring 2023).

Our best candidates are the smaller versions of the best performing open source models:
- 8 Bn parameters version of LLaMA
- 7 Bn parameters version of [LLaMA](https://arxiv.org/pdf/2302.13971.pdf) - best for spring 2023, released by Facebook
- 7 Bn parameters version of [Falcon](https://falconllm.tii.ae) - close competitor to Llama, released in May 2023 by [Technology Innovation Institute of UAE](https://www.tii.ae).
- 6.7 Bn parameters version of [OPT](https://arxiv.org/abs/2205.01068) - top choice in this nomination in 2022, released by Facebook.

Beware: while these models are smaller than the ones in API, they're still over 60x larger than the BERT we played with last time. The code below will *just barely* fit into memory, so make sure you don't have anything else loaded. Sometimes you may need to restart runtime for the code to work.

It's a good time to restart your kernel and switch to GPU! (Runtime -> Change runtime type)
<center><img src="https://i.imgur.com/OOfDYzJ.png" width=240px></center>

In [ ]:
#%pip install --upgrade transformers bitsandbytes accelerate sentencepiece optimum auto-gptq
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

If you want to use `'meta-llama/Llama-3.2-3B',` you should add the huggungface token. We use a free option

In [ ]:
# FOR SECRET ON KAGGLE KERNAL
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# FOR SECRET ON COLAB
# from google.colab import userdata
# secret_value_0 = userdata.get('HUGGINGFACE_TOKEN')

#Another option:
# from huggingface_hub import notebook_login
# notebook_login()

Below is an example of a simple pipeline, that you don't want in general to use. That's why it's commented

In [ ]:
# from transformers import pipeline
# model_id = 'huggyllama/llama-7b' # "meta-llama/Llama-3.2-3B"

# pipe = pipeline(
#     "text-generation",
#     model=model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     token=secret_value_0
# )

# pipe("The key to life is")

## Text generation

**Comparison of strategies for language model text generation:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # token=secret_value_0
)

You can use different generation strategies

In [ ]:
from transformers import GenerationConfig

In [ ]:
# Define generation config
generation_config = GenerationConfig(
    max_length=200,
    temperature=0.8,
    top_p=0.9,
    do_sample=True
)

# greedy inference:                                        do_sample=False)
# beam search for highest probability:                     num_beams=4)

In [ ]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))

### You can use quntized model's weights (the following code does the quantization on the fly)

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True, #or load_in_8bit=True
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4"
#     )

In [ ]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,  #old way: load_in_8bit=True
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    # token=secret_value_0
    )

### Or you can upload a quantized model

In [ ]:
model_name = "TheBloke/Llama-2-13B-GPTQ" #meta-llama/Llama-3-8B-GPTQ

tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision="main")

In [ ]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))
#print(tokenizer.decode(output[0], skip_special_tokens=True))

#### Low-level code for text generation

In [ ]:
# You may need to clear GPU memory
# import gc
# del output_tokens
# del model
# del tokenizer
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True
)

In [ ]:
prompt = "New York is the capital of"
# prompt = "Skippy, a young android, likes to dream about electric"

print(prompt, '\n')

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}  # reverse vocab for decode

for i in range(10):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    prompt += next_token

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f"Step #{i} candidates:")
    for t, p in zip (top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t:<10}: {p:.4f} ")

    print(f'\nChosen token: {next_token}', end='\n\n', flush=True)

### Part 3: Chain-of-thought prompting (4 points total)

![img](https://github.com/kojima-takeshi188/zero_shot_cot/raw/main/img/image_stepbystep.png)

---



In [ ]:
import json
import random
import locale; locale.getpreferredencoding = lambda: "UTF-8"
!wget https://raw.githubusercontent.com/kojima-takeshi188/zero_shot_cot/2824685e25809779dbd36900a69825068e9f51ef/dataset/AQuA/test.json -O aqua.json
data = list(map(json.loads, open("aqua.json")))

In [ ]:
print("Example:")
data[150]

### Naive solution

Here, we prompt the model to choose an answer to the example above (`data[150]`) out of the options given above. We're using a format that mimics grade school solution textbook.

Please note that there are minor formatting changes in options: an extra space and an opening bracket. Those may or may not be important :)

In [ ]:
EXAMPLE_0SHOT = """
Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Correct Answer:
""".strip()

In [ ]:
# solving an equation directly
batch = tokenizer(EXAMPLE_0SHOT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_0SHOT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))

And here's how you can solve this with few-shot chain-of-thought prompting.

You need to chang 3 things
- use a new field called **Rationale**, that contains a step-by-step solution to the problem
- add several few-shot examples of previously solved problems **with rationales**
- change the final prompt so that the model has to generate rationale before answering

In [ ]:
EXAMPLE_3SHOT_CHAIN_OF_THOUGHT = """
Question: The original retail price of an appliance was 60 percent more than its wholesale cost. If the appliance was actually sold for 20 percent less than the original retail price, then it was sold for what percent more than its wholesale cost?
Answer Choices: (A) 20% (B) 28% (C) 36% (D) 40% (E) 42%
Rationale: wholesale cost = 100;\noriginal price = 100*1.6 = 160;\nactual price = 160*0.8 = 128.\nAnswer: B.
Correct Answer: B


Question: A grocer makes a 25% profit on the selling price for each bag of flour it sells. If he sells each bag for $100 and makes $3,000 in profit, how many bags did he sell?
Answer Choices: (A) 12 (B) 16 (C) 24 (D) 30 (E) 40
Rationale: Profit on one bag: 100*1.25= 125\nNumber of bags sold = 3000/125 = 24\nAnswer is C.
Correct Answer: C


Question: 20 marbles were pulled out of a bag of only white marbles, painted black, and then put back in. Then, another 20 marbles were pulled out, of which 1 was black, after which they were all returned to the bag. If the percentage of black marbles pulled out the second time represents their percentage in the bag, how many marbles in total Q does the bag currently hold?
Answer Choices: (A) 40 (B) 200 (C) 380 (D) 400 (E) 3200
Rationale: We know that there are 20 black marbles in the bag and this number represent 1/20 th of the number of all marbles in the bag, thus there are total Q of 20*20=400 marbles.\nAnswer: D.
Correct Answer: D


Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Rationale:
""".strip()

In [ ]:
batch = tokenizer(EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_3SHOT_CHAIN_OF_THOUGHT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))
#### NOTE: scroll down for the final answer (below the ======= line)

__Task 5 (3 pt)__ write a function that automatically creates chain-of-thought prompts. Follow the instructions from the function docstring.

In [ ]:
QUESTION_PREFIX = "Question: "
OPTIONS_PREFIX = "Answer Choices: "
CHAIN_OF_THOUGHT_PREFIX = "Rationale: "
ANSWER_PREFIX = "Correct Answer: "
FEWSHOT_SEPARATOR = "\n\n\n"

def make_prompt(*, main_question, fewshot_examples):
  """
  Your goal is to produce the same prompt as the EXAMPLE_3SHOT_CHAIN_OF_THOUGHT automatically

  For each few-shot question, make sure to follow the following rules:
  1. Each question begins with QUESTION_PREFIX, after which you should print the question without leading/traiiling spaces (if any)
  2. After the question, provide space-separated options. Each option should be put in double brackets, followed by option text, e.g. "(A) 146%"
  3. Then, provide the answer as a single letter (A-E)
  4. Finally, add trailing newlines from FEWSHOT_SEPARATOR

  Your final prompt should contain all fewshot_examples (in order), separated with FEWSHOT_SEPARATOR, then follow with main_question.
  The main_question should contain the question and options formatted the same way as in FEWSHOT_EXAMPLES.
  After that, you should prompt the model to produce an explanation (rationale) for the answer.

  Please make sure your prompt contains no leading/trailing newlines or spaces, same as in EXAMPLE_3SHOT_CHAIN_OF_THOUGHT
  """

  def format_question(q):
    question = q["question"]
    options = q["options"]
    rationale = q["rationale"]
    correct = q["correct"]

    i = 0
    options_str = ''
    for option in options:
      option_str = f"({option[0]}) {option[2:]}"
      
      if i < len(options) - 1:
        option_str += ' '
      i += 1
      
      options_str += option_str
  
    return f"{QUESTION_PREFIX}{question}\n{OPTIONS_PREFIX}{options_str}\n{CHAIN_OF_THOUGHT_PREFIX}{rationale}\n{ANSWER_PREFIX}{correct}{FEWSHOT_SEPARATOR}"
  
  fewshot_examples_str = ''
  for example in fewshot_examples:
    fewshot_examples_str += format_question(example)
  
  main_question_str = main_question["question"]
  main_options = main_question["options"]
  
  options_str = ''
  i = 0
  for option in main_options:
    option_str = f"({option[0]}) {option[2:]}"
    
    if i < len(main_options) - 1:
      option_str += ' '
    i += 1
    
    options_str += option_str

  main_options_str = f"{QUESTION_PREFIX}{main_question_str}\n{OPTIONS_PREFIX}{options_str}\n{CHAIN_OF_THOUGHT_PREFIX}"

  return f"{fewshot_examples_str}{main_options_str}".strip()
    
  
generated_fewshot_prompt = make_prompt(main_question=data[150], fewshot_examples=(data[30], data[20], data[5]))
assert generated_fewshot_prompt == EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, "prompts don't match"
assert generated_fewshot_prompt != make_prompt(main_question=data[150], fewshot_examples=())
assert generated_fewshot_prompt.endswith(make_prompt(main_question=data[150], fewshot_examples=()))

print("Well done!")

# Hint: if two prompts do not match, you may find it usefull to use https://www.diffchecker.com or similar to find the difference

__Task 6 (3 points):__ Evaluate your prompt.

Please run the model on the entire dataset and measure it's accuracy.
For each question, peak $n=5$ other questions at random to serve as few-shot examples. Make sure not to accidentally sample the main_question among few-shot examples. For scientific evaluation, it is also a good practice to split the data into two parts: one for eval, and another for few-shot examples. However, doing so is optional in this homework.

The tricky part is when to stop generating: if you don't control for this, your model can accidentally generate a whole new question - and promptyly answer it :) To make sure you get the correct answer, stop generating tokens when the model is done explaining it's solution. To circumvent this, you need to __stop generating as soon as the model generates Final Answer: [A-E]__
To do so, you can either generate manually (see low-level generation above) or use [transformers stopping criteria](https://discuss.huggingface.co/t/implimentation-of-stopping-criteria-list/20040/2), whichever you prefer.

If you do everything right, the model should be much better than random. However, please __do not expect miracles__: this is far from the best models, and it will perform much worse than an average human.

In [ ]:
NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0    # how many times did the model's chosen answer (letter) match the correct answer

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
from tqdm import tqdm

class AnswerStoppingCriteria(StoppingCriteria):
    def __init__(self, start_length, tokenizer):
        self.start_length = start_length
        self.tokenizer = tokenizer

    def __call__(self, input_ids, scores, **kwargs):
        # Decode ONLY the newly generated tokens
        generated_text = self.tokenizer.decode(input_ids[0][self.start_length:], skip_special_tokens=True)
        return any(f"Correct Answer: {ch}" in generated_text for ch in ['A', 'B', 'C', 'D', 'E'])

def evaluate_model(n_fewshot, tries, data, pre_prend=""):
    global NUM_SAMPLES, NUM_RESPONDED, NUM_CORRECT
    progress_bar = tqdm(data, desc="Evaluating", dynamic_ncols=True)
    
    for main_question in progress_bar:
        NUM_SAMPLES += 1
        reties = 0
        while reties < tries:
            fewshot_examples = random.sample([q for q in data if q != main_question], n_fewshot)
            prompt = f"{pre_prend}{FEWSHOT_SEPARATOR}{make_prompt(main_question=main_question, fewshot_examples=fewshot_examples)}".strip()
            inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

            stopping_criteria = AnswerStoppingCriteria(start_length=inputs['input_ids'].shape[1], tokenizer=tokenizer)

            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                stopping_criteria=StoppingCriteriaList([stopping_criteria]),
                do_sample=True,
                top_k=5,
                top_p=0.95,
                temperature=0.7,
            )

            generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            predicted_answer = generated_text.strip().split('Correct Answer: ')[-1].split()[0]
            if predicted_answer in ['A', 'B', 'C', 'D', 'E']:
                reties = tries
                NUM_RESPONDED += 1
                if predicted_answer == main_question["correct"]:
                    NUM_CORRECT += 1
            else:
                reties += 1
    
        progress_bar.set_postfix({
            "Samples": NUM_SAMPLES,
            "Responded": NUM_RESPONDED,
            "Correct": NUM_CORRECT,
            "Acc": f"{(NUM_CORRECT / NUM_RESPONDED * 100):.1f}%" if NUM_RESPONDED else "N/A",
            "Response Rate": f"{(NUM_RESPONDED / NUM_SAMPLES * 100):.1f}%" if NUM_SAMPLES else "N/A",
        })

evaluate_model(5, 3, data)

print("num_samples:", NUM_SAMPLES)
print("num_responded:", NUM_RESPONDED)
print("num_correct:", NUM_CORRECT)

In [ ]:
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)

if NUM_RESPONDED / NUM_SAMPLES < 0.9:
  print("Something is wrong with the evaluation technique (for 5-shot CoT): the model refuses to answer too many questions.")
  print("Make sure you generate enough tokens that the model can produce a correct answer.")
  print("When in doubt, take a look at the full model output. You can often spot errors there.")

__Task 7 (2 points)__ Experiment time!
<img width=200px src=https://www.evolvefish.com/cdn-cgi/image/quality%3D85/assets/images/Apparel/TShirtsWomenCont/Main/EF-APP-CWT-00068(Main).jpg>

Your final quest is to use the testbench you've just written to answer one of the following questions:

### Option 1: How many shots do you need?

How does model accuracy change with the number of fewshot examples?

a. check if the model accuracy changes as you increase/decrease the number of "shots"

b. try to prompt-engineer a model into giving the best rationale __without__ any few-shot examples, i.e. zero-shot

For zero-shot mode, feel free to use wild prompt-engineering or modify the inference procedure.

### Option 2: Is this prompting tecnique reliable?

_Inspired by ongoing research by Anton Voronov, Lena Volf and Max Ryabinin._

For this option, you need to check if the model behavior (and hence, accuracy) is robust to perturbations in the input prompt.

a. Does the accuracy degrade if you provide wrong answers to few-shot examples? (make sure to modify rationale if it contains answer in the end)

b. Does it degrade if you replace question/answer prompts with "Q" and "A"? What if you write both on the same line? Change few-shot separators?



### Option 3: Inference Matters

There are many ways to inference the model, not all of them equal.

a. check whether greedy inference or beam search affects model generation quality

b. implement and evaluate sampling with voting (see explanation below).


The voting technique(b) should work as follows: first, you generate k (e.g. 50) "attempts" at an answer using nucleus sampling (or a similar technique).
Then, you count how many of those attempts chose a particular option (A, B, etc) as the final answer. The option that was chosen most frequently has the most "votes", and therefore "wins".

To speed up voting, you may want to generate these attempts in parallel as a batch. That should be very easy to implement: just run `model.generate` on a list with multiple copies of the same prompt.




================================================

__Common rules:__ You will need to test both hypothes (A and B) in the chosen option. You may choose to replace one of them with your own idea - but please ask course staff in advance (via telegram) if you want full points.

Feel free to organize your code and report as you see fit - but please make sure it's readable and the code runs top-to-bottom :)
Write a short informal report about what you tried and, in doing so, what did you found. Minimum of 2 paragraphs; more is ok; creative visualizations are welcome.

You are allowed (but not required) to prompt the model into generating a report for you --- or helping you write one. However, if you do so, make sure that it is still human-readable :)



In [ ]:
# feel free to organize your solution as you see it
def print_stats():
    print("num_samples:", NUM_SAMPLES)
    print("num_responded:", NUM_RESPONDED)
    print("num_correct:", NUM_CORRECT)

    print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
    print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
    print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)
    
def reset_stats():
    global NUM_SAMPLES, NUM_RESPONDED, NUM_CORRECT
    NUM_SAMPLES = 0
    NUM_RESPONDED = 0
    NUM_CORRECT = 0
    
zero_shot_pre_promt = """
You are a helpful AI that solves multiple choice math word problems. For each question, think step-by-step to find the correct solution. Then, choose the correct answer letter from the choices.
Show your reasoning under "Rationale:", then finish by stating the final answer like this: Correct Answer: B""".strip()
n_shot_count = [1,3,5,7]
data_subset = data

print("-----------------Zero-shot evaluation-----------------")
reset_stats()
evaluate_model(0, 2, data_subset, pre_prend=zero_shot_pre_promt)
print_stats()

for n_shot in n_shot_count:
    print(f"-----------------{n_shot}-shot evaluation-----------------")
    reset_stats()
    evaluate_model(n_shot, 2, data_subset)
    print_stats()


#Option 1:


# Report

for option one i decided to try both 0 shot with a promt to tell it how to answer followed by multi-shot promts from 1 - 7

for the zero shot promts we see the mode struggels to even respond, out of all my tester prompts the best one that outlines exactly what the model should do and 3 attempts to give a respons we still see the modle to strugle to give an answe 20% of the time

as for the multi shot prompts we see that the response rate returns back up to >90% when giver 3 tries. as for accuracy we see that as we add more example prompts the model get better pettering of at 5 as 7 and 10 are not as big of a diffrence